# SeisBench Testcase Prediction (Detection/Picking) On Earthquake Swarms in the PNSN Network
This notebook provides a worked example of using pre-trained ML models in SeisBench using data from two earthquake swarms within the PNSN network:
1. the May 2017 earthquake swarm near Bremerton  
        - **Note** this example uses just 2 days of the swarm: May 10-11, 2017. Catalog searches show the swarm can broadly be defined between May 8 - July 
2. the August 23-24, 2013 earthquake swarm near Mount St. Helens

#### Attribution
- Code structure based on discussions with Yiyu Ni (University of Washington)  
- "Peppy seismic swarm 20 km NW of Mount St. Helens" blog post by John Vidale on August 24, 2013  
https://pnsn.org/blog/2013/08/24/peppy-seismic-swarm-20-km-nw-of-mount-st-helens
- "Earthquake swarm NE of Bremerton" blog post by Renate Hartog on May 11, 2017  
https://www.pnsn.org/blog/2017/05/11/earthquake-swarm-ne-of-bremerton

In [7]:
# Import dependencies
from seisbench.models import EQTransformer, PhaseNet
from obspy import UTCDateTime
import torch
import pandas as pd
import matplotlib.pyplot as plt
import os
%matplotlib



In [18]:
# Load PNSN catalog data from custom queries for Bremerton-Adjacent Swarm (BRM)
df_BRM_full = pd.read_csv(os.path.join('data','2017-05-10_BRM','pnsn_event_catalog_2017_BRM_cluster.csv'),\
                          parse_dates=['Time UTC'],index_col='Time UTC')
# Sort by index
df_BRM_full = df_BRM_full.sort_index()

print(df_BRM_full)

                         Evid  Magnitude Magnitude Type    Epoch(UTC)  \
Time UTC                                                                
2017-01-25 14:03:20  61230427        0.9              l  1.485353e+09   
2017-01-25 14:03:57  60132008        0.2              d  1.485353e+09   
2017-01-25 17:03:40  61230447        1.0              l  1.485364e+09   
2017-03-15 17:57:25  61257761        1.2              l  1.489601e+09   
2017-05-03 19:20:36  61277931        3.3              l  1.493839e+09   
...                       ...        ...            ...           ...   
2018-04-06 14:40:31  61377731        1.6              l  1.523026e+09   
2018-04-10 14:36:39  61378401        0.5              d  1.523371e+09   
2018-04-24 05:00:49  61365397        0.9              l  1.524546e+09   
2018-05-06 19:21:33  61370492        1.0              l  1.525634e+09   
2018-05-08 15:27:01  61371382        1.5              l  1.525793e+09   

                                  Time Local      

In [31]:
# Plot the catalog
fig = plt.figure(figsize=(10,10))
axa = fig.add_subplot(211)
axa.plot(df_BRM_full['Magnitude'],'.')
axa.set_xlabel('UTC Date Time')
axa.set_ylabel('Magnitude (mixed methods)')
axb = axa.twinx()
axb.plot(df_BRM_full['Magnitude'].cumsum(),'r-',alpha=0.5)
axb.set_ylabel('Cumulative magnitude',labelpad=15)

axc = fig.add_subplot(223)
ch = axc.scatter(df_BRM_full['Lon'],df_BRM_full['Lat'],s = 5**df_BRM_full['Magnitude'],c=df_BRM_full.index)
# plt.colorbar(ax=axc,cax=ch)

Using matplotlib backend: <object object at 0x13518a880>


RuntimeError: No mappable was found to use for colorbar creation. First define a mappable such as an image (with imshow) or a contour set (with contourf).

### Data selection
Use the following fields to define an event and station query
 - **TS**: [obspy.UTCDateTime] Start time of query in UTC
 - **TE**:      [obspy.UTCDateTime] End time of query in UTC
- **lat0**:     [float] centroid latitude for queries in *degrees North*
 - **lon0**:    [float] centroid longitude for queries in *degrees East*
 - **Eradius**: [float] radius for Earthquake query in *meters*
 - **Sradius**: [float] radius for Station query in *meters*

In [4]:
# Define fields for query
fields = ('TS','TE','lat0','lon0','Eradius','Sradius')
# Define data selection kwargs 
dict_BRM = dict(zip(fields,[UTCDateTime("2017-05-10T00:00:00"),\
                            UTCDateTime("2017-05-11T23:59:59"),\
                            47.5828,-122.57841,\
                            7500.,50e3]))

# dict_MSH = dict(zip(fields,[]))

# Set desired query key-word arguments for subsequent blocks
q_kwargs = dict_BRM
# Display parameters
print('Selected query parameters')
print(q_kwargs)


Selected query parameters
{'TS': UTCDateTime(2017, 5, 10, 0, 0), 'TE': UTCDateTime(2017, 5, 11, 23, 59, 59), 'lat0': 47.5828, 'lon0': -122.57841, 'Eradius': 8000.0, 'Sradius': 50000.0}


In [ ]:
# Use IRIS webservices / FDSN to query station, event, and waveform data 
